In [1]:
import re
import json

M3U_FILE = "tung_iptv.m3u"
OUTPUT_JSON = "tung_iptv.json"

PROVIDER_ID = "tung-iptv"
PROVIDER_NAME = "Tung IPTV"
PROVIDER_URL = "https://example.com/tung-iptv"
PROVIDER_COLOR = "#1E90FF"
PROVIDER_DESC = "Tung IPTV - Live channels"

FALLBACK_IMAGE = "https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/images/logo.png"


def slugify(text):
    return re.sub(r'[^a-z0-9]+', '-', text.lower()).strip('-')


def image_obj(url=None, display="cover", w=640, h=480):
    return {
        "url": url or FALLBACK_IMAGE,
        "display": display,
        "width": w,
        "height": h
    }


def parse_m3u(path):
    groups = {}
    current = None

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()

            if not line or line.startswith("#EXTM3U"):
                continue

            if line.startswith("#EXTINF"):
                group = re.search(r'group-title="([^"]+)"', line)
                logo = re.search(r'tvg-logo="([^"]+)"', line)
                name = line.split(",", 1)[-1].strip()

                current = {
                    "group": group.group(1) if group else "Others",
                    "name": name,
                    "logo": logo.group(1) if logo else None
                }

            elif current and not line.startswith("#"):
                group_name = current["group"]
                groups.setdefault(group_name, [])

                cid = slugify(current["name"])

                channel = {
                    "id": cid,
                    "name": current["name"],
                    "type": "single",
                    "display": "thumbnail-only",
                    "image": image_obj(current["logo"]),
                    "sources": [
                        {
                            "id": f"{cid}-source",
                            "name": "Live",
                            "contents": [
                                {
                                    "id": f"{cid}-content",
                                    "name": current["name"],
                                    "streams": [
                                        {
                                            "id": f"{cid}-stream",
                                            "name": "Live",
                                            "remote_data": {
                                                "url": line
                                            }
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }

                groups[group_name].append(channel)
                current = None

    return groups


def build_json(groups):
    return {
        "id": PROVIDER_ID,
        "url": PROVIDER_URL,
        "name": PROVIDER_NAME,
        "color": PROVIDER_COLOR,
        "description": PROVIDER_DESC,
        "image": {
            "url": FALLBACK_IMAGE
        },
        "groups": [
            {
                "id": slugify(group_name),
                "name": group_name,
                "display": "vertical",
                "channels": channels
            }
            for group_name, channels in groups.items()
        ],
        "notice": {
            "id": "notice",
            "link": "https://t.me/yourchannel",
            "icon": FALLBACK_IMAGE,
            "closeable": True
        },
        "option": {
            "save_history": False,
            "save_search_history": False,
            "save_wishlist": False
        }
    }


if __name__ == "__main__":
    groups = parse_m3u(M3U_FILE)
    data = build_json(groups)

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ SUCCESS: {len(groups)} groups written to {OUTPUT_JSON}")

✅ SUCCESS: 4 groups written to tung_iptv.json
